In [7]:
from langchain.vectorstores.redis import Redis
from langchain.embeddings import OpenAIEmbeddings

#embeddings = OpenAIEmbeddings()

metadata = [
    {
        "user": "john",
        "age": 18,
        "job": "engineer",
        "credit_score": "high",
    },
    {
        "user": "derrick",
        "age": 45,
        "job": "doctor",
        "credit_score": "low",
    },
    {
        "user": "nancy",
        "age": 94,
        "job": "doctor",
        "credit_score": "high",
    },
    {
        "user": "tyler",
        "age": 100,
        "job": "engineer",
        "credit_score": "high",
    },
    {
        "user": "joe",
        "age": 35,
        "job": "dentist",
        "credit_score": "medium",
    },
]

texts = ["foo", "foo", "foo", "bar", "bar"]


Redis.__init__()



rds = Redis.from_texts(
    texts,
    embeddings, 
    metadatas=metadata,
    redis_url="redis://localhost:6379",
    index_name="users",
)

rds.index_name

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)